In [3]:
import numpy as np
from pathlib import Path
import re
from tqdm import tqdm
from skimage import io
from skimage.morphology import skeletonize_3d, binary_dilation
from scipy.ndimage import distance_transform_edt
import tifffile as tif
from scipy.ndimage import binary_fill_holes 
import cc3d
from scipy.io import loadmat, savemat
import skan
import networkx as nx
from collections import Counter
import pandas as pd

In [4]:
directory = Path('james_preds')
files  = directory.glob('*_warped_radii.pickle')
files = sorted([x.as_posix() for x in files])
files[0]

'james_preds/XYZres006_0001_warped_radii.pickle'

In [10]:
for i in tqdm(range(len(files))):
    graph = nx.read_gpickle(files[i])
    deg_1_nodes = [node for (node, val) in graph.degree() if val == 1]
    for j in deg_1_nodes:
        edge = list(graph.edges(j))[0]
        if graph[edge[0]][edge[1]]['weight'] < 15:
            graph.remove_edge(edge[0],edge[1])
            if edge[0] in deg_1_nodes:
                deg_1_nodes.remove(edge[0])
            if edge[1] in deg_1_nodes:
                deg_1_nodes.remove(edge[1])
    #deg_1_nodes = [node for (node, val) in graph.degree() if val == 1]
    #deg_1_edges = [x for x in list(graph.edges) if any(val in x for val in deg_1_nodes)]
    #index1=0
    #finished = []
    #while index1 < len(deg_1_edges):
    #    deg_1_nodes = [node for (node, val) in graph.degree() if val == 1]
    #    deg_1_edges = [x for x in list(graph.edges) if any(val in x for val in deg_1_nodes) and x not in finished]
    #    j = deg_1_edges[index1]
    #    index2 = 0
    #    while index2 < len(deg_1_edges):
    #        k = deg_1_edges[index2]
    #        index2 += 1
    #        if index2 == len(deg_1_edges):
    #            index1 += 1
    #            finished.append(j)
    #        if np.all(j != k):
    #            path0 = graph[j[0]][j[1]]['pts']
    #            path1 = graph[k[0]][k[1]]['pts']
    #            index = 5
    #            if graph.degree(j[0]) == 1:
    #                new_vertex_1 = j[0]
    #            else:
    #                new_vertex_1 = j[1]
    #            if graph.degree(k[0]) == 1:
    #                new_vertex_2 = k[0]
    #            else:
    #                new_vertex_2 = k[1]
    #            if np.all(np.abs(path0[-1] - path1[0]) <= index ):
    #                graph.add_edge(new_vertex_1,new_vertex_2)
    #                graph[new_vertex_1][new_vertex_2]['weight'] = graph[j[0]][j[1]]['weight'] + graph[k[0]][k[1]]['weight']
    #                graph[new_vertex_1][new_vertex_2]['pts'] = np.concatenate(
    #                    (graph[j[0]][j[1]]['pts'],
    #                     graph[k[0]][k[1]]['pts'][:]), 
    #                     axis=0)
    #                graph[new_vertex_1][new_vertex_2]['path_weights'] = np.concatenate(
    #                    (graph[j[0]][j[1]]['path_weights'],
    #                     graph[k[0]][k[1]]['path_weights'][:]), 
    #                    axis=0)
    #                graph[new_vertex_1][new_vertex_2]['radii'] = np.mean(graph[new_vertex_1][new_vertex_2]['path_weights'])
    #                graph[new_vertex_1][new_vertex_2]['radii_std'] = np.std(graph[new_vertex_1][new_vertex_2]['path_weights'])
    #                graph[new_vertex_1][new_vertex_2]['end-0'] = graph[new_vertex_1][new_vertex_2]['pts'][0]
    #                graph[new_vertex_1][new_vertex_2]['end-1'] = graph[new_vertex_1][new_vertex_2]['pts'][-1]
    #                graph.remove_edge(j[0],j[1])
    #                graph.remove_edge(k[0],k[1])
    #                deg_1_nodes = [node for (node, val) in graph.degree() if val == 1]
    #                deg_1_edges = [x for x in list(graph.edges) if any(val in x for val in deg_1_nodes)]
    #                index1 = 0
    #                break
    #            elif np.all(np.abs(path0[-1] - path1[-1]) <= index ):
    #                graph.add_edge(new_vertex_1,new_vertex_2)
    #                graph[new_vertex_1][new_vertex_2]['weight'] = graph[j[0]][j[1]]['weight'] + graph[k[0]][k[1]]['weight']
    #                graph[new_vertex_1][new_vertex_2]['pts'] = np.concatenate(
    #                    (graph[j[0]][j[1]]['pts'],
    #                     graph[k[0]][k[1]]['pts'][::-1][:]), 
    #                     axis=0)
    #                graph[new_vertex_1][new_vertex_2]['path_weights'] = np.concatenate(
    #                    (graph[j[0]][j[1]]['path_weights'],
    #                     graph[k[0]][k[1]]['path_weights'][::-1][:]), 
    #                    axis=0)
    #                graph[new_vertex_1][new_vertex_2]['radii'] = np.mean(graph[new_vertex_1][new_vertex_2]['path_weights'])
    #                graph[new_vertex_1][new_vertex_2]['radii_std'] = np.std(graph[new_vertex_1][new_vertex_2]['path_weights'])
    #                graph[new_vertex_1][new_vertex_2]['end-0'] = graph[new_vertex_1][new_vertex_2]['pts'][0]
    #                graph[new_vertex_1][new_vertex_2]['end-1'] = graph[new_vertex_1][new_vertex_2]['pts'][-1]
    #                graph.remove_edge(j[0],j[1])
    #                graph.remove_edge(k[0],k[1])
    #                deg_1_nodes = [node for (node, val) in graph.degree() if val == 1]
    #                deg_1_edges = [x for x in list(graph.edges) if any(val in x for val in deg_1_nodes)]
    #                index1 = 0
    #                break
    #            elif np.all(np.abs(path0[0] - path1[0]) <= index ):
    #                graph.add_edge(new_vertex_1,new_vertex_2)
    #                graph[new_vertex_1][new_vertex_2]['weight'] = graph[j[0]][j[1]]['weight'] + graph[k[0]][k[1]]['weight']
    #                graph[new_vertex_1][new_vertex_2]['pts'] = np.concatenate(
    #                    (graph[j[0]][j[1]]['pts'][::-1],
    #                     graph[k[0]][k[1]]['pts'][:]), 
    #                     axis=0)
    #                graph[new_vertex_1][new_vertex_2]['path_weights'] = np.concatenate(
    #                    (graph[j[0]][j[1]]['path_weights'],
    #                     graph[k[0]][k[1]]['path_weights'][::-1][:]), 
    #                    axis=0)
    #                graph[new_vertex_1][new_vertex_2]['radii'] = np.mean(graph[new_vertex_1][new_vertex_2]['path_weights'])
    #                graph[new_vertex_1][new_vertex_2]['radii_std'] = np.std(graph[new_vertex_1][new_vertex_2]['path_weights'])
    #                graph[new_vertex_1][new_vertex_2]['end-0'] = graph[new_vertex_1][new_vertex_2]['pts'][0]
    #                graph[new_vertex_1][new_vertex_2]['end-1'] = graph[new_vertex_1][new_vertex_2]['pts'][-1]
    #                graph.remove_edge(j[0],j[1])
    #                graph.remove_edge(k[0],k[1])
    #                deg_1_nodes = [node for (node, val) in graph.degree() if val == 1]
    #                deg_1_edges = [x for x in list(graph.edges) if any(val in x for val in deg_1_nodes)]
    #                index1 = 0
    #                break
    #            elif np.all(np.abs(path0[0] - path1[-1]) <= index ):
    #                graph.add_edge(new_vertex_1,new_vertex_2)
    #                graph[new_vertex_1][new_vertex_2]['weight'] = graph[j[0]][j[1]]['weight'] + graph[k[0]][k[1]]['weight']
    #                graph[new_vertex_1][new_vertex_2]['pts'] = np.concatenate(
    #                    (graph[j[0]][j[1]]['pts'][::-1],
    #                     graph[k[0]][k[1]]['pts'][::-1][:]), 
    #                     axis=0)
    #                graph[new_vertex_1][new_vertex_2]['path_weights'] = np.concatenate(
    #                    (graph[j[0]][j[1]]['path_weights'],
    #                     graph[k[0]][k[1]]['path_weights'][::-1][:]), 
    #                    axis=0)
    #                graph[new_vertex_1][new_vertex_2]['radii'] = np.mean(graph[new_vertex_1][new_vertex_2]['path_weights'])
    #                graph[new_vertex_1][new_vertex_2]['radii_std'] = np.std(graph[new_vertex_1][new_vertex_2]['path_weights'])
    #                graph[new_vertex_1][new_vertex_2]['end-0'] = graph[new_vertex_1][new_vertex_2]['pts'][0]
    #                graph[new_vertex_1][new_vertex_2]['end-1'] = graph[new_vertex_1][new_vertex_2]['pts'][-1]
    #                graph.remove_edge(j[0],j[1])
    #                graph.remove_edge(k[0],k[1])
    #                deg_1_nodes = [node for (node, val) in graph.degree() if val == 1]
    #                deg_1_edges = [x for x in list(graph.edges) if any(val in x for val in deg_1_nodes)]
    #                index1 = 0
    #                break
    #    deg_1_nodes = [node for (node, val) in graph.degree() if val == 1]
    #    deg_1_edges = [x for x in list(graph.edges) if any(val in x for val in deg_1_nodes) and x not in finished]
    deg_1_nodes = [node for (node, val) in graph.degree() if val == 1]                
    for j in deg_1_nodes:
        edge = list(graph.edges(j))[0]
        if graph[edge[0]][edge[1]]['weight'] < 10:
            graph.remove_edge(edge[0],edge[1])
            if edge[0] in deg_1_nodes:
                deg_1_nodes.remove(edge[0])
            if edge[1] in deg_1_nodes:
                deg_1_nodes.remove(edge[1])
    deg_2_nodes = [node for (node, val) in graph.degree() if val == 2]
    while len(deg_2_nodes) > 0:
        new_edge = tuple(graph.neighbors(deg_2_nodes[0]))
        if len(new_edge) == 1:
            graph.remove_node(deg_2_nodes[0])
            deg_2_nodes = [node for (node, val) in graph.degree() if val == 2]
        else:
            graph.add_edge(*tuple(graph.neighbors(deg_2_nodes[0])))
            graph[new_edge[0]][new_edge[1]]['weight'] = graph[new_edge[0]][deg_2_nodes[0]]['weight'] + graph[new_edge[1]][deg_2_nodes[0]]['weight']
            if np.all(graph[new_edge[0]][deg_2_nodes[0]]['pts'][-1] == graph[new_edge[1]][deg_2_nodes[0]]['pts'][0]):
                graph[new_edge[0]][new_edge[1]]['pts'] = np.concatenate(
                    (graph[new_edge[0]][deg_2_nodes[0]]['pts'],
                     graph[new_edge[1]][deg_2_nodes[0]]['pts'][1:]), 
                     axis=0)
                graph[new_edge[0]][new_edge[1]]['path_weights'] = np.concatenate(
                    (graph[new_edge[0]][deg_2_nodes[0]]['path_weights'],
                     graph[new_edge[1]][deg_2_nodes[0]]['path_weights'][1:]), 
                    axis=0)
            elif np.all(graph[new_edge[0]][deg_2_nodes[0]]['pts'][-1] == graph[new_edge[1]][deg_2_nodes[0]]['pts'][-1]):
                graph[new_edge[0]][new_edge[1]]['pts'] = np.concatenate(
                    (graph[new_edge[0]][deg_2_nodes[0]]['pts'],
                     graph[new_edge[1]][deg_2_nodes[0]]['pts'][::-1][1:]), 
                     axis=0)
                graph[new_edge[0]][new_edge[1]]['path_weights'] = np.concatenate(
                    (graph[new_edge[0]][deg_2_nodes[0]]['path_weights'],
                     graph[new_edge[1]][deg_2_nodes[0]]['path_weights'][::-1][1:]), 
                    axis=0)
            elif np.all(graph[new_edge[0]][deg_2_nodes[0]]['pts'][0] == graph[new_edge[1]][deg_2_nodes[0]]['pts'][0]):
                graph[new_edge[0]][new_edge[1]]['pts'] = np.concatenate(
                    (graph[new_edge[0]][deg_2_nodes[0]]['pts'][::-1],
                     graph[new_edge[1]][deg_2_nodes[0]]['pts'][1:]), 
                     axis=0)
                graph[new_edge[0]][new_edge[1]]['path_weights'] = np.concatenate(
                    (graph[new_edge[0]][deg_2_nodes[0]]['path_weights'],
                     graph[new_edge[1]][deg_2_nodes[0]]['path_weights'][::-1][1:]), 
                    axis=0)
            elif np.all(graph[new_edge[0]][deg_2_nodes[0]]['pts'][0] == graph[new_edge[1]][deg_2_nodes[0]]['pts'][-1]):
                graph[new_edge[0]][new_edge[1]]['pts'] = np.concatenate(
                    (graph[new_edge[0]][deg_2_nodes[0]]['pts'][::-1],
                     graph[new_edge[1]][deg_2_nodes[0]]['pts'][::-1][1:]), 
                     axis=0)
                graph[new_edge[0]][new_edge[1]]['path_weights'] = np.concatenate(
                    (graph[new_edge[0]][deg_2_nodes[0]]['path_weights'],
                     graph[new_edge[1]][deg_2_nodes[0]]['path_weights'][::-1][1:]), 
                    axis=0)
            else:
                match = False
                index = 1
                while match == False:
                    if np.all(np.abs(graph[new_edge[0]][deg_2_nodes[0]]['pts'][-1] - graph[new_edge[1]][deg_2_nodes[0]]['pts'][0]) <= index ):
                        graph[new_edge[0]][new_edge[1]]['pts'] = np.concatenate(
                            (graph[new_edge[0]][deg_2_nodes[0]]['pts'],
                             graph[new_edge[1]][deg_2_nodes[0]]['pts'][:]), 
                             axis=0)
                        graph[new_edge[0]][new_edge[1]]['path_weights'] = np.concatenate(
                            (graph[new_edge[0]][deg_2_nodes[0]]['path_weights'],
                             graph[new_edge[1]][deg_2_nodes[0]]['path_weights'][:]), 
                            axis=0)
                        match = True
                    elif np.all(np.abs(graph[new_edge[0]][deg_2_nodes[0]]['pts'][-1] - graph[new_edge[1]][deg_2_nodes[0]]['pts'][-1]) <= index ):
                        graph[new_edge[0]][new_edge[1]]['pts'] = np.concatenate(
                            (graph[new_edge[0]][deg_2_nodes[0]]['pts'],
                             graph[new_edge[1]][deg_2_nodes[0]]['pts'][::-1][:]), 
                             axis=0)
                        graph[new_edge[0]][new_edge[1]]['path_weights'] = np.concatenate(
                            (graph[new_edge[0]][deg_2_nodes[0]]['path_weights'],
                             graph[new_edge[1]][deg_2_nodes[0]]['path_weights'][::-1][:]), 
                            axis=0)
                        match = True
                    elif np.all(np.abs(graph[new_edge[0]][deg_2_nodes[0]]['pts'][0] - graph[new_edge[1]][deg_2_nodes[0]]['pts'][0]) <= index ):
                        graph[new_edge[0]][new_edge[1]]['pts'] = np.concatenate(
                            (graph[new_edge[0]][deg_2_nodes[0]]['pts'][::-1],
                             graph[new_edge[1]][deg_2_nodes[0]]['pts'][:]), 
                             axis=0)
                        graph[new_edge[0]][new_edge[1]]['path_weights'] = np.concatenate(
                            (graph[new_edge[0]][deg_2_nodes[0]]['path_weights'],
                             graph[new_edge[1]][deg_2_nodes[0]]['path_weights'][::-1][:]), 
                            axis=0)
                        match = True
                    elif np.all(np.abs(graph[new_edge[0]][deg_2_nodes[0]]['pts'][0] - graph[new_edge[1]][deg_2_nodes[0]]['pts'][-1]) <= index ):
                        graph[new_edge[0]][new_edge[1]]['pts'] = np.concatenate(
                            (graph[new_edge[0]][deg_2_nodes[0]]['pts'][::-1],
                             graph[new_edge[1]][deg_2_nodes[0]]['pts'][::-1][:]), 
                             axis=0)
                        graph[new_edge[0]][new_edge[1]]['path_weights'] = np.concatenate(
                            (graph[new_edge[0]][deg_2_nodes[0]]['path_weights'],
                             graph[new_edge[1]][deg_2_nodes[0]]['path_weights'][::-1][:]), 
                            axis=0)
                        match = True
                    index += 1
            graph[new_edge[0]][new_edge[1]]['radii'] = np.mean(graph[new_edge[0]][new_edge[1]]['path_weights'])
            graph[new_edge[0]][new_edge[1]]['radii_std'] = np.std(graph[new_edge[0]][new_edge[1]]['path_weights'])
            graph[new_edge[0]][new_edge[1]]['end-0'] = graph[new_edge[0]][new_edge[1]]['pts'][0]
            graph[new_edge[0]][new_edge[1]]['end-1'] = graph[new_edge[0]][new_edge[1]]['pts'][-1]
            graph.remove_node(deg_2_nodes[0])
            deg_2_nodes = [node for (node, val) in graph.degree() if val == 2]
    nx.write_gpickle(graph,re.sub('.pickle','_clean.pickle',files[i]))

100%|██████████| 94/94 [00:07<00:00, 12.20it/s]


In [6]:
re.sub('.pickle','_clean.pickle',files[i])

'james_preds/XYZres052_warped_radii_clean.pickle'

In [22]:
k

(44, 89)

In [15]:
len(deg_1_edges)

248

In [ ]:
graph[new_vertex_1][new_vertex_2]['pts'] = np.concatenate(
         (graph[j[0]][j[1]]['pts'],
          graph[k[0]][k[1]]['pts'][:]), 
          axis=0)

In [ ]:
graph[new_vertex_1][9]['pts']

In [ ]:
graph[new_vertex_2][10]['pts']

In [ ]:
len(graph.edges)

In [ ]:
graph[list(graph.neighbors(j))[0]][j]

In [ ]:
while len(deg_2_nodes) > 0:
    new_edge = tuple(graph.neighbors(deg_2_nodes[0]))
    if len(new_edge) == 1:
        graph.remove_node(deg_2_nodes[0])
        deg_2_nodes = [node for (node, val) in graph.degree() if val == 2]
    else:
        graph.add_edge(*tuple(graph.neighbors(deg_2_nodes[0])))
        graph[new_edge[0]][new_edge[1]]['weight'] = graph[new_edge[0]][deg_2_nodes[0]]['weight'] + graph[new_edge[1]][deg_2_nodes[0]]['weight']
        if graph[new_edge[0]][deg_2_nodes[0]]['pts'][-1].all() == graph[new_edge[1]][deg_2_nodes[0]]['pts'][0].all():
            graph[new_edge[0]][new_edge[1]]['pts'] = np.concatenate(
                (graph[new_edge[0]][deg_2_nodes[0]]['pts'],
                 graph[new_edge[1]][deg_2_nodes[0]]['pts'][1:]), 
                 axis=0)
            graph[new_edge[0]][new_edge[1]]['path_weights'] = np.concatenate(
                (graph[new_edge[0]][deg_2_nodes[0]]['path_weights'],
                 graph[new_edge[1]][deg_2_nodes[0]]['path_weights'][1:]), 
                axis=0)
        elif graph[new_edge[0]][deg_2_nodes[0]]['pts'][-1].all() == graph[new_edge[1]][deg_2_nodes[0]]['pts'][-1].all():
            graph[new_edge[0]][new_edge[1]]['pts'] = np.concatenate(
                (graph[new_edge[0]][deg_2_nodes[0]]['pts'],
                 graph[new_edge[1]][deg_2_nodes[0]]['pts'][::-1][1:]), 
                 axis=0)
            graph[new_edge[0]][new_edge[1]]['path_weights'] = np.concatenate(
                (graph[new_edge[0]][deg_2_nodes[0]]['path_weights'],
                 graph[new_edge[1]][deg_2_nodes[0]]['path_weights'][::-1][1:]), 
                axis=0)
        else:
            error
        graph[new_edge[0]][new_edge[1]]['radii'] = np.mean(graph[new_edge[0]][new_edge[1]]['path_weights'])
        graph[new_edge[0]][new_edge[1]]['radii_std'] = np.std(graph[new_edge[0]][new_edge[1]]['path_weights'])
        graph[new_edge[0]][new_edge[1]]['end-0'] = graph[new_edge[0]][new_edge[1]]['pts'][0]
        graph[new_edge[0]][new_edge[1]]['end-0'] = graph[new_edge[0]][new_edge[1]]['pts'][-1]
        graph.remove_node(deg_2_nodes[0])
        deg_2_nodes = [node for (node, val) in graph.degree() if val == 2]

In [ ]:
deg_2_nodes

In [ ]:
graph[new_edge[0]][deg_2_nodes[0]]['pts']

In [ ]:
graph[new_edge[1]][deg_2_nodes[0]]['pts']

In [ ]:
np.concatenate((graph[new_edge[0]][deg_2_nodes[0]]['pts'],graph[new_edge[1]][deg_2_nodes[0]]['pts'][1:]), axis=0)

In [ ]:
graph[new_edge[1]][new_edge[0]]['radii']

In [ ]:
graph[0]

In [ ]:
deg_2_nodes = [node for (node, val) in graph.degree() if val == 2]

In [ ]:
deg_2_nodes

In [ ]:
graph.degree(j[0])